##### The aim of this project to classify reddit comments in three distinct labels: Verterinarian, Medical Doctor and Others. The project will take place in two steps: First step would be the creation of labels for every comment gathered from the reddit platform ,utilizing the gpt 3.5 turbo LLM  and the secon step would involve the training and evaluation of created models utilizing the pretrained BERT transformer models.

In [33]:
#Import libraries
import pandas as pd
import psycopg2 as py
import os
from dotenv import load_dotenv
pd.set_option('display.max_rows', None)

In [34]:
load_dotenv()

True

Fisrt we import our data utilizing the psycopg2 library which provides us with a platform to connect remotely to the postgresql database.

In [36]:
#Connecting to the database/server using psycopg2
conn=py.connect(os.environ.get('CONNECTION_STRING'))

In [4]:
#Selecting all comments from the reddit_usernames_comments table
sql_query_comments='SELECT * FROM reddit_usernames_comments'

In [5]:
#Reading in data using pandas
reddit_comments=pd.read_sql(sql_query_comments, conn)

C:\Users\jessica.ogwu\AppData\Local\Temp\ipykernel_36600\822161584.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reddit_comments=pd.read_sql(sql_query_comments, conn)


In [6]:
#Saving data collected in a csv file for easier retrieval and data manipulation
reddit_comments.to_csv('reddit_comments.csv')


In [7]:
#Reading in data to a pandas dataframe
df = pd.read_csv('reddit_comments.csv', usecols=range(1, 3))
df.head()

,username,comments
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on..."
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ..."
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...
3,abarthch,"I see of course there are changing variables, ..."
4,VoodooKing,I have 412+ and faced issues because wireguard...


Carrying out some Exploratory Data Analysis on data

In [9]:
#Length of dataframe
print(f'The size of the dataframe is', len(df))

The size of the dataframe is 3276


In [10]:
df.shape

(3276, 2)

The dataframe contains 3276 rows and 2 columns

In [11]:
#Some descriptive statistics of dataframe
df.describe()

,username,comments
count,3275,3276
unique,3275,3275
top,LoveAGoodTwist,Why?|Why?|Why?|Why?|Why?|Why?
freq,1,2


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  3275 non-null   object
 1   comments  3276 non-null   object
dtypes: object(2)
memory usage: 51.3+ KB


Dataframe seems to not contain null values

In [15]:
#Next we view some comments to check for the need for extra preprocessing
#Viewing one comment and copying out output to check for errors and possibly diagnose the need for extra cleaning
df['comments'][0]

'Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a wee

'Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.|Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.'

Comment above shows duplication of one comment multiple times divided by pipes ('|'). It will thus be necessry to clean comments and select distinct comment from concatenation of duplicates.

### Preprocessing on the comments column

In [16]:
from collections import OrderedDict

def process_comments(comment):
    # Split the comment by pipe and remove duplicates while preserving order
    comment_list = comment.split('|')
    unique_comments = list(OrderedDict.fromkeys(comment.strip() for comment in comment_list))
    joined_comments = ' '.join(unique_comments)
    return joined_comments  # Returns the list of unique comments

# Apply the function and observe the order-preserving behavior
df['processed_comments'] = df['comments'].apply(process_comments)

In [17]:
#Viewing our newly processed dataframe
df.head()

,username,comments,processed_comments
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on...","Female, Kentucky. 4 years out. Work equine on..."
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ...","As a woman of reproductive age, fuck Texas"
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...,what makes you want to become a vet?
3,abarthch,"I see of course there are changing variables, ...","I see of course there are changing variables, ..."
4,VoodooKing,I have 412+ and faced issues because wireguard...,I have 412+ and faced issues because wireguard...


In [18]:
df['processed_comments'][0]

'Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.'

'Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.'

Data seems to be clean and ready to go.

As stated above, we begin the first step which is classifying comments into various three labels.

In [170]:
#Import libraries
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import nest_asyncio

In [171]:
load_dotenv(find_dotenv(), override=True)

True

Next, we will call the openAI API on 10 rows to test the classification of comments into:
1. Verterinarian
2. Medical Doctor
3. Others

In [244]:
import aiohttp
import asyncio
from cachetools import TTLCache
import os  # Ensure os is imported for accessing environment variables
import pandas as pd  # Assuming pandas is imported for DataFrame operations

# Apply nested asyncio patches, required for certain environments like Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

# Create a TTL (Time-To-Live) cache to store API results for 300 seconds
cache = TTLCache(maxsize=1000, ttl=300)

# Asynchronous function to fetch classification from OpenAI API
async def fetch_classification(session, comment, retry_count=0):
    # Check if the result is already in cache
    if comment in cache:
        return cache[comment]
    
    # Prepare headers for the API request
    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json"
    }
    
    # Prepare the payload with the prompt for classification
    payload = {
        "model": "gpt-3.5-turbo",
        "max_tokens": 50,
        "messages": [{
            "role": "user", 
            "content": (
                f"Classify the profession based on this statement: '{comment}'. Consider the following categories:\n"
                "- Medical Doctor: Typically involves humans in a clinical or hospital setting.\n"
                "- Veterinarian: Involves animal health care, including veterinarians and related professions.\n"
                "- Others: Any profession not directly related to medical or veterinary fields.\n"
                "Provide a reason for your classification." 
                "Strictly follow the response format like this 'category'|'reason'."
            )
        }]
    }
    
    try:
        # Make the API request
        async with session.post('https://api.openai.com/v1/chat/completions', headers=headers, json=payload) as response:
            result = await response.json()  # Parse the JSON response
            
            # Handle API errors and retry if necessary
            if 'error' in result:
                if retry_count < 5:
                    await asyncio.sleep(20)  # Wait before retrying
                    return await fetch_classification(session, comment, retry_count + 1)
            
            # Extract content from the API response
            content = result['choices'][0]['message']['content']
            cache[comment] = content  # Cache the result
            return content
    except Exception as e:
        # Retry in case of exceptions
        if retry_count < 5:
            await asyncio.sleep(20)
            return await fetch_classification(session, comment, retry_count + 1)
        else:
            return "Failed to classify due to errors", "Repeated errors occurred."

# Asynchronous function to classify multiple comments
async def classify_comments(comments):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_classification(session, comment) for comment in comments]  # Create a list of tasks
        return await asyncio.gather(*tasks)  # Run tasks concurrently and gather results

# Copy the DataFrame (assuming df is already defined)
df2 = df.copy()

# Extract the first 10 processed comments to classify
comments = df2['processed_comments'].head(10).tolist()
loop = asyncio.get_event_loop()
results = loop.run_until_complete(classify_comments(comments))  # Run the classification

# Function to process results from the API and split them into label and reason
def process_results(results):
    processed_results = []
    for res in results:
        if isinstance(res, tuple):  # Handle error cases
            processed_results.append(("Error", res[0]))
        elif '|' in res:
            processed_results.append(res.split('|'))  # Split valid responses into label and reason
        else:
            processed_results.append(("Unclassified", "Format error or missing '|' in response"))  # Handle format errors
    return processed_results

# Apply the processing function and create new columns in the DataFrame
df2[['label', 'reason']] = pd.DataFrame(process_results(results), index=df2.head(10).index)


In [245]:
#Viewing results 
df2.head(10)

,username,comments,processed_comments,label,reason
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on...","Female, Kentucky. 4 years out. Work equine on...",Veterinarian,The statement mentions working in equine only ...
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ...","As a woman of reproductive age, fuck Texas",Others,The statement does not pertain to medical or v...
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...,what makes you want to become a vet?,Veterinarian,The statement specifically mentions the desire...
3,abarthch,"I see of course there are changing variables, ...","I see of course there are changing variables, ...",Others,The statement does not mention anything relate...
4,VoodooKing,I have 412+ and faced issues because wireguard...,I have 412+ and faced issues because wireguard...,Others,The individual is discussing the setup and con...
5,queerofengland,"Contrary to employers' belief, at will does no...","Contrary to employers' belief, at will does no...",Veterinarian,The statements focus primarily on issues relat...
6,tikitessie,He makes us laugh every day :) favorite cowork...,He makes us laugh every day :) favorite coworker,Others,The statement suggests that the person's profe...
7,theophania808,I've worked in the vet industry for years and ...,I've worked in the vet industry for years and ...,Veterinarian,The statement mentions working in the vet indu...
8,waterpencilboop,Wow. Those techs are really rude! It sounds li...,Wow. Those techs are really rude! It sounds li...,Others,The statement does not mention anything relate...
9,paxbanana0,"Those are long, probably stressful days at wor...","Those are long, probably stressful days at wor...",Veterinarian,The statement mentions recommending dental wor...


Relatively good output. However, due to API rate limit constraint, couldn't call the function on all 3000+ rows. Went ahead to pass only the first 1000 rows.

In [247]:
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
import os
from cachetools import TTLCache

nest_asyncio.apply()
cache = TTLCache(maxsize=1000, ttl=300)

async def fetch_classification(session, comment, retry_count=0):
    if comment in cache:
        return cache[comment]
    
    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gpt-3.5-turbo",
        "max_tokens": 50,
        "messages": [{
            "role": "user", 
            "content": (
                f"Classify the profession based on this statement: '{comment}'. Consider the following categories:\n"
                "- Medical Doctor: Typically involves humans in a clinical or hospital setting.\n"
                "- Veterinarian: Involves animal health care, including veterinarians and related professions.\n"
                "- Others: Any profession not directly related to medical or veterinary fields.\n"
                "Provide a reason for your classification." 
                "Strictly follow the response format like this 'category'|'reason'."
            )
        }]
    }
    try:
        async with session.post('https://api.openai.com/v1/chat/completions', headers=headers, json=payload) as response:
            result = await response.json()
            if 'error' in result:
                if retry_count < 5:
                    await asyncio.sleep(20)
                    return await fetch_classification(session, comment, retry_count + 1)
            content = result['choices'][0]['message']['content']
            cache[comment] = content
            return content
    except Exception as e:
        if retry_count < 5:
            await asyncio.sleep(20)
            return await fetch_classification(session, comment, retry_count + 1)
        else:
            return "Failed to classify due to errors", "Repeated errors occurred."

async def classify_comments(comments):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_classification(session, comment) for comment in comments]
        return await asyncio.gather(*tasks)

def process_results(results):
    processed_results = []
    for res in results:
        if isinstance(res, tuple):
            processed_results.append(("Error", res[0]))
        elif '|' in res:
            processed_results.append(res.split('|'))
        else:
            processed_results.append(("Unclassified", "Format error or missing '|' in response"))
    return processed_results

def process_in_chunks(dataframe, chunk_size=10):
    final_df = pd.DataFrame()
    for start in range(0, dataframe.shape[0], chunk_size):
        end = start + chunk_size
        subset = dataframe.iloc[start:end]
        comments = subset['processed_comments'].tolist()
        loop = asyncio.get_event_loop()
        results = loop.run_until_complete(classify_comments(comments))
        processed_data = process_results(results)
        temp_df = pd.DataFrame(processed_data, columns=['label', 'reason'], index=subset.index)
        final_df = pd.concat([final_df, temp_df])
    return final_df

# Apply the function to the top 1000 DataFrame
df2 = df.iloc[0:1000]
final_results_df = process_in_chunks(df2)



In [254]:
#Creating a new df with label and reason
new_df = df.iloc[0:1000]
new_df[['label', 'reason']] = final_results_df

/var/folders/kl/wfsfkbsn3y71j3wwn8b25pzr0000gn/T/ipykernel_85240/2636324001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['label', 'reason']] = final_results_df
/var/folders/kl/wfsfkbsn3y71j3wwn8b25pzr0000gn/T/ipykernel_85240/2636324001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['label', 'reason']] = final_results_df


In [258]:
#First 5 rows in new df
new_df.head()

,username,comments,processed_comments,label,reason
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on...","Female, Kentucky. 4 years out. Work equine on...",Category: Veterinarian,The statement mentions working in equine-only...
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ...","As a woman of reproductive age, fuck Texas",Others,The statement does not specifically mention an...
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...,what makes you want to become a vet?,Veterinarian,The question specifically mentions becoming a ...
3,abarthch,"I see of course there are changing variables, ...","I see of course there are changing variables, ...",Others,The statement does not relate to medical or ve...
4,VoodooKing,I have 412+ and faced issues because wireguard...,I have 412+ and faced issues because wireguard...,Others,The statement provided does not indicate any d...


From above, some comments seemto have inconsistent labels, like 'category:Veterinarian', next, we clean up for consistency.

In [259]:
# Standardize the labels
def standardize_labels(label):
    label = label.lower().replace("'", "").strip()
    if 'veterinarian' in label:
        return 'Veterinarian'
    elif 'doctor' in label or 'medical' in label:
        return 'Medical Doctor'
    else:
        return 'Others'

new_df['standardized_label'] = new_df['label'].apply(standardize_labels)

/var/folders/kl/wfsfkbsn3y71j3wwn8b25pzr0000gn/T/ipykernel_85240/4238548593.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['standardized_label'] = new_df['label'].apply(standardize_labels)


In [261]:
#Getting the counts of each labels
new_df.standardized_label.value_counts()

standardized_label
Others            851
Veterinarian      140
Medical Doctor      9
Name: count, dtype: int64

In [262]:
result_df = new_df[['username','processed_comments','standardized_label','reason']]
result_df.head()

,username,processed_comments,standardized_label,reason
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on...",Veterinarian,The statement mentions working in equine-only...
1,wahznooski,"As a woman of reproductive age, fuck Texas",Others,The statement does not specifically mention an...
2,Churro_The_fish_Girl,what makes you want to become a vet?,Veterinarian,The question specifically mentions becoming a ...
3,abarthch,"I see of course there are changing variables, ...",Others,The statement does not relate to medical or ve...
4,VoodooKing,I have 412+ and faced issues because wireguard...,Others,The statement provided does not indicate any d...


Due to dataset imbalance, let's generate synthetic data for catgorizations with Medical Doctor to match the amount of data we have for Veterinarian.

- Total rows for Veterinarian is 140.
- Total rows for Medical Doctor is 9.

131 synthetic data for Medical Doctor is required to match that of Veterinarian.

In [385]:
import openai

# Initialize the OpenAI client using your API key (ensure this key is set in your environment)
client = openai

# Define the chat completion request to generate synthetic data for medical practitioners
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": (
                # Prompt to generate synthetic data
                "Generate 131 examples of sentences that a medical doctor might say about their field on reddit social network.\n"
                "Ensure the max token allows for all 131 sentences to be created."
            )
        }
    ],
    model="gpt-3.5-turbo",  # Specify the model to use
)

# The result of the API call is stored in the 'chat_completion' variable


In [386]:
#Results
json_string = chat_completion.choices[0].message.content
print(json_string)

1. As a doctor, I recommend getting regular check-ups to maintain good health.
2. It's important to stay up-to-date on the latest advances in medical research.
3. Always consult with a healthcare professional before making any major decisions about your health.
4. Did you know that the average person should drink at least 8 glasses of water a day?
5. High blood pressure can lead to serious health problems if left untreated.
6. It's crucial to listen to your body and seek medical attention when something feels off.
7. Maintaining a healthy diet and regular exercise routine can significantly improve your overall health.
8. Medical science has come a long way in the past century, with new treatments and medications constantly being developed.
9. I always encourage my patients to ask questions and be proactive about their healthcare.
10. It's never too late to make positive changes to your lifestyle for better health outcomes.
11. Remember to get enough sleep each night to allow your body 

In [389]:
# Converting output to a pandas dataframe
comments = [line.split(". ", 1)[1] for line in json_string.strip().split("\n")]

# Create a DataFrame
synthetic_df = pd.DataFrame(comments, columns=['processed_comments'])
synthetic_df.head()

,processed_comments
0,"As a doctor, I recommend getting regular check..."
1,It's important to stay up-to-date on the lates...
2,Always consult with a healthcare professional ...
3,Did you know that the average person should dr...
4,High blood pressure can lead to serious health...


In [392]:
#Generate columns similar to those of real data
synthetic_df['username'] = ['synthetic_' + str(i) for i in synthetic_df.index]
synthetic_df['standardized_label'] = 'Medical Doctor'
synthetic_df['reason'] = 'Synthetic Data'

synthetic_df = synthetic_df[['username', 'processed_comments', 'standardized_label', 'reason']]
synthetic_df.head()

,username,processed_comments,standardized_label,reason
0,synthetic_0,"As a doctor, I recommend getting regular check...",Medical Doctor,Synthetic Data
1,synthetic_1,It's important to stay up-to-date on the lates...,Medical Doctor,Synthetic Data
2,synthetic_2,Always consult with a healthcare professional ...,Medical Doctor,Synthetic Data
3,synthetic_3,Did you know that the average person should dr...,Medical Doctor,Synthetic Data
4,synthetic_4,High blood pressure can lead to serious health...,Medical Doctor,Synthetic Data


In [394]:
#Concatenate synthetic data with original data and check counts
concat_df = pd.concat([result_df, synthetic_df]).reset_index(drop=True)
concat_df.standardized_label.value_counts()

standardized_label
Others            851
Veterinarian      140
Medical Doctor    140
Name: count, dtype: int64

In [395]:
concat_df.to_csv('balanced_data.csv', index=False)

Open model_with_synthetic_data.ipynb